In [15]:
import numpy as np
import pandas as pd
from scipy.stats import gumbel_r, weibull_min, lognorm

# --- Parameters for the Dataset ---
# User requested 10,000 data points
N_TIMESTEPS = 10000
# The following values are adjusted to be consistent with the new N_TIMESTEPS
TOTAL_DURATION_HOURS = N_TIMESTEPS * 1 / 3600  # Matches the new N_TIMESTEPS
TIMESTEP_SECONDS = 1  # 1 second sampling rate

RANDOM_STATE = 42

# Leak event parameters
# User requested 15% of data points to be leaks
LEAK_RATIO = 0.15
N_LEAK_TIMESTEPS_TOTAL = int(N_TIMESTEPS * LEAK_RATIO)
N_LEAK_EVENTS = 5 # Number of distinct leak events to simulate
MIN_LEAK_DURATION_MINUTES = 5
MAX_LEAK_DURATION_MINUTES = 60

# Slugging event parameters (normal operational transients)
N_SLUGGING_EVENTS = 10 # Number of distinct slugging events
MIN_SLUGGING_DURATION_MINUTES = 10
MAX_SLUGGING_DURATION_MINUTES = 30
SLUGGING_AMPLITUDE_FACTOR = 0.15 # % of normal value for fluctuations

np.random.seed(RANDOM_STATE)

# --- Define realistic ranges and distributions based on research for MULTIPHASE PIPELINES ---

# General operational parameters
TEMP_NORMAL_LOC = 60 # C - Typical wellhead temperature [3, 4]
TEMP_NORMAL_SCALE = 5 # C - Variability around normal temperature
TEMP_LEAK_DROP_MAX = 5 # C - Subtle temperature drop for multiphase leaks (less pronounced than pure gas) [5, 6]

# Multiphase pipeline parameters (combining oil and gas characteristics)
PRESS_LOC = 700 # psi - Mid-range for multiphase gathering/transmission [7, 8]
PRESS_SCALE = 50 # psi - Base variability in pressure
PRESS_SLUG_FLUCTUATION_FACTOR = 0.2 # % of current pressure for slugging fluctuation [9]

# Oil flow parameters
BOPD_MEAN = 665 # barrel of oil produced per day (exp(6.5) from prev. lognormal mean) [10]
BOPD_SCALE = 100 # Variability in BOPD

# Water cut (ratio of produced water to oil) [11]
WATER_CUT_MIN = 3 # Minimum produced water to oil ratio (>3 to >20)
WATER_CUT_MAX = 10 # Maximum produced water to oil ratio

# Gas-Oil Ratio (GOR) in scf/bbl. Light oil range: 300 < GOR <= 100,000 scf/bbl [12]
GOR_MIN = 500 # scf/bbl
GOR_MAX = 5000 # scf/bbl

# Gas composition and properties
CO2_MOL_MIN = 0.19 # mol % - Minimum CO2 in pipeline quality gas [13, 14]
CO2_MOL_MAX = 3.0 # mol % - Maximum CO2 in pipeline quality gas (spec limit) [14]
GAS_GRAV_LOC = 0.65 # dimensionless - Typical gas specific gravity (0.60-0.70) [15, 16]
GAS_GRAV_SCALE = 0.03 # Variability in gas specific gravity

# Basic Sediment & Water (BS&W) for the liquid phase [17, 18]
BSW_MIN = 0.1 # %
BSW_MAX = 2.5 # %

# Corrosion defect parameters (normalized depth, 0 to 1 where 1 is through-wall)
# Based on research, corrosion depth has significant impact on failure [19, 20, 21]
CORROSION_NO_LEAK_LOC = 0.05 # Mean for minor defects (no leak)
CORROSION_NO_LEAK_SCALE = 0.01 # Scale for minor defects
CORROSION_LEAK_LOC = 0.4 # Mean for significant defects (leading to leak)
CORROSION_LEAK_SCALE = 0.1 # Scale for significant defects

# Leak severity thresholds for CR-corrosion defect (normalized depth) [19, 21]
CRITICAL_DEFECT_SMALL_LEAK = 0.2 # Threshold for a small leak
CRITICAL_DEFECT_LARGE_LEAK = 0.35 # Threshold for a large leak
CRITICAL_DEFECT_RUPTURE = 0.6 # Threshold for a rupture

# Impact of leak on features (as a percentage reduction of normal values)
LEAK_PRESSURE_DROP_MAX_PERCENT = 0.5 # Up to 50% pressure drop for rupture [22, 8]
LEAK_FLOW_DEVIATION_MAX_PERCENT = 0.3 # Up to 30% flow deviation for rupture [23, 24]

# --- Initialize time series arrays ---
time_stamps = np.arange(0, N_TIMESTEPS * TIMESTEP_SECONDS, TIMESTEP_SECONDS)
wellhead_temp = np.zeros(N_TIMESTEPS)
wellhead_press = np.zeros(N_TIMESTEPS)
mmcfd_gas = np.zeros(N_TIMESTEPS)
bopd = np.zeros(N_TIMESTEPS)
bwpd = np.zeros(N_TIMESTEPS)
bsw = np.zeros(N_TIMESTEPS)
co2_mol = np.zeros(N_TIMESTEPS)
gas_grav = np.zeros(N_TIMESTEPS)
corrosion_defect = np.zeros(N_TIMESTEPS)
leak_status = np.array(['No Leak'] * N_TIMESTEPS, dtype=object)

# --- Generate Baseline Data with Diurnal/Seasonal Trends and Noise ---
# Simulate a subtle diurnal temperature cycle
daily_cycle = np.sin(time_stamps / (24 * 3600) * 2 * np.pi) * 5 # +/- 5C daily swing
wellhead_temp_baseline = TEMP_NORMAL_LOC + daily_cycle + np.random.normal(0, TEMP_NORMAL_SCALE / 5, N_TIMESTEPS)

# Simulate baseline BOPD with some variability
bopd_baseline = np.random.lognormal(mean=np.log(BOPD_MEAN), sigma=BOPD_SCALE / BOPD_MEAN, size=N_TIMESTEPS)
bopd_baseline = np.maximum(bopd_baseline, 10)

# Simulate baseline GOR and Water Cut
gor_baseline = np.random.uniform(GOR_MIN, GOR_MAX, N_TIMESTEPS)
water_cut_baseline = np.random.uniform(WATER_CUT_MIN, WATER_CUT_MAX, N_TIMESTEPS)

# Calculate baseline gas and water production
mmcfd_gas_baseline = (bopd_baseline * gor_baseline) / 1_000_000
mmcfd_gas_baseline = np.maximum(mmcfd_gas_baseline, 0.01)

bwpd_baseline = bopd_baseline * water_cut_baseline

# Simulate baseline CO2 and Gas Grav
co2_mol_baseline = np.random.uniform(CO2_MOL_MIN, CO2_MOL_MAX, N_TIMESTEPS)
gas_grav_baseline = np.clip(np.random.normal(loc=GAS_GRAV_LOC, scale=GAS_GRAV_SCALE / 5, size=N_TIMESTEPS), 0.60, 0.70)

# Simulate baseline BSW
bsw_baseline = np.random.uniform(BSW_MIN, BSW_MAX, N_TIMESTEPS)

# Baseline pressure correlated with temp and total flow
total_flow_equivalent_baseline = bopd_baseline + (mmcfd_gas_baseline * 100) # Simple scaling for correlation
wellhead_press_baseline = PRESS_LOC + (wellhead_temp_baseline * 1.0) + (total_flow_equivalent_baseline * 0.01) + \
                          np.random.normal(0, PRESS_SCALE / 5, N_TIMESTEPS)
wellhead_press_baseline = np.maximum(wellhead_press_baseline, 10)

# Initialize all features with baseline values
wellhead_temp[:] = wellhead_temp_baseline
wellhead_press[:] = wellhead_press_baseline
mmcfd_gas[:] = mmcfd_gas_baseline
bopd[:] = bopd_baseline
bwpd[:] = bwpd_baseline
bsw[:] = bsw_baseline
co2_mol[:] = co2_mol_baseline
gas_grav[:] = gas_grav_baseline
corrosion_defect[:] = np.random.normal(loc=CORROSION_NO_LEAK_LOC, scale=CORROSION_NO_LEAK_SCALE, size=N_TIMESTEPS)
corrosion_defect = np.clip(corrosion_defect, 0, CRITICAL_DEFECT_SMALL_LEAK - 0.001)

# --- Introduce Slugging Events (Normal Transients) ---
# Keep track of indices already used for events
used_indices = set()
for _ in range(N_SLUGGING_EVENTS):
    slug_start_idx = np.random.randint(0, N_TIMESTEPS - MAX_SLUGGING_DURATION_MINUTES * 60)
    slug_duration_seconds = np.random.randint(MIN_SLUGGING_DURATION_MINUTES * 60, MAX_SLUGGING_DURATION_MINUTES * 60)
    slug_end_idx = min(slug_start_idx + slug_duration_seconds, N_TIMESTEPS)

    slug_indices = np.arange(slug_start_idx, slug_end_idx)

    # Check for overlap with existing events and re-choose if necessary
    if not used_indices.isdisjoint(slug_indices):
        continue
    
    used_indices.update(slug_indices)

    # Simulate pressure fluctuations due to slugging
    slug_noise_pressure = np.random.normal(0, wellhead_press_baseline[slug_indices] * PRESS_SLUG_FLUCTUATION_FACTOR, len(slug_indices))
    wellhead_press[slug_indices] += slug_noise_pressure

    # Simulate flow fluctuations due to slugging
    slug_noise_bopd = np.random.normal(0, bopd_baseline[slug_indices] * SLUGGING_AMPLITUDE_FACTOR, len(slug_indices))
    bopd[slug_indices] += slug_noise_bopd
    mmcfd_gas[slug_indices] += np.random.normal(0, mmcfd_gas_baseline[slug_indices] * SLUGGING_AMPLITUDE_FACTOR, len(slug_indices))
    bwpd[slug_indices] += np.random.normal(0, bwpd_baseline[slug_indices] * SLUGGING_AMPLITUDE_FACTOR, len(slug_indices))

    # Ensure no negative values after slugging fluctuations
    wellhead_press[slug_indices] = np.maximum(wellhead_press[slug_indices], 10)
    bopd[slug_indices] = np.maximum(bopd[slug_indices], 0)
    mmcfd_gas[slug_indices] = np.maximum(mmcfd_gas[slug_indices], 0)
    bwpd[slug_indices] = np.maximum(bwpd[slug_indices], 0)


# --- Introduce Leak Events with exact ratio ---
leak_timesteps_count = 0
num_leak_events_generated = 0
while leak_timesteps_count < N_LEAK_TIMESTEPS_TOTAL:
    # Ensure there's enough space for a leak event
    remaining_timesteps = N_TIMESTEPS - leak_timesteps_count
    if remaining_timesteps < MIN_LEAK_DURATION_MINUTES * 60:
        break
        
    leak_start_idx = np.random.randint(0, N_TIMESTEPS)
    
    # Check if a random starting point is already in a used index
    if leak_start_idx in used_indices:
        continue
        
    leak_duration_seconds = np.random.randint(MIN_LEAK_DURATION_MINUTES * 60, MAX_LEAK_DURATION_MINUTES * 60)
    
    # Cap the duration to not exceed the total required leak timesteps
    leak_duration_seconds = min(leak_duration_seconds, N_LEAK_TIMESTEPS_TOTAL - leak_timesteps_count)
    
    leak_end_idx = min(leak_start_idx + leak_duration_seconds, N_TIMESTEPS)

    leak_indices_for_event = np.arange(leak_start_idx, leak_end_idx)

    # Check for overlap with existing events
    if not used_indices.isdisjoint(leak_indices_for_event):
        continue

    # All checks passed, apply the event
    used_indices.update(leak_indices_for_event)
    leak_timesteps_count += len(leak_indices_for_event)
    num_leak_events_generated += 1
    
    # Determine leak type and corresponding defect severity
    leak_type_choice = np.random.choice(['Small Leak', 'Large Leak', 'Rupture'], p=[0.5, 0.3, 0.2])
    
    defect_val = 0
    if leak_type_choice == 'Small Leak':
        defect_val = np.random.uniform(CRITICAL_DEFECT_SMALL_LEAK, CRITICAL_DEFECT_LARGE_LEAK - 0.001)
    elif leak_type_choice == 'Large Leak':
        defect_val = np.random.uniform(CRITICAL_DEFECT_LARGE_LEAK, CRITICAL_DEFECT_RUPTURE - 0.001)
    else: # Rupture
        defect_val = np.random.uniform(CRITICAL_DEFECT_RUPTURE, 1.0)

    # Apply leak effects over the duration
    for t_idx in leak_indices_for_event:
        # Gradual onset for small leaks, more sudden for large/ruptures [25, 26]
        onset_progress = (t_idx - leak_start_idx) / (leak_duration_seconds * 0.1) # 10% of duration for onset
        onset_factor = min(1.0, onset_progress)

        # Scale leak impact factor based on defect severity and onset
        leak_impact_factor = (defect_val - CRITICAL_DEFECT_SMALL_LEAK) / (1.0 - CRITICAL_DEFECT_SMALL_LEAK)
        current_impact = leak_impact_factor * onset_factor

        # Apply pressure drop [25, 27, 22, 28, 29, 8, 30]
        wellhead_press[t_idx] -= wellhead_press_baseline[t_idx] * LEAK_PRESSURE_DROP_MAX_PERCENT * current_impact
        
        # Apply flow deviation [31, 32, 23, 33, 34, 1, 26]
        bopd[t_idx] -= bopd_baseline[t_idx] * LEAK_FLOW_DEVIATION_MAX_PERCENT * current_impact
        mmcfd_gas[t_idx] -= mmcfd_gas_baseline[t_idx] * LEAK_FLOW_DEVIATION_MAX_PERCENT * current_impact
        bwpd[t_idx] -= bwpd_baseline[t_idx] * LEAK_FLOW_DEVIATION_MAX_PERCENT * current_impact
        
        # Apply subtle temperature drop [5, 6]
        wellhead_temp[t_idx] -= TEMP_LEAK_DROP_MAX * current_impact

        # Update corrosion defect and leak status for the leak period
        corrosion_defect[t_idx] = defect_val
        leak_status[t_idx] = leak_type_choice

# Ensure no negative flow/pressure values after leak effects
wellhead_press = np.maximum(wellhead_press, 10) # Minimum realistic pressure [25, 7]
mmcfd_gas = np.maximum(mmcfd_gas, 0)
bopd = np.maximum(bopd, 0)
bwpd = np.maximum(bwpd, 0)

# --- Combine all features into a DataFrame ---
df = pd.DataFrame({
    'Timestamp': time_stamps,
    'Wellhead_Temp. (C)': wellhead_temp,
    'Wellhead_Press (psi)': wellhead_press,
    'MMCFD- gas': mmcfd_gas,
    'BOPD (barrel of oil produced per day)': bopd,
    'BWPD (barrel of water produced per day)': bwpd,
    'BSW basic solid and water(%)': bsw,
    'CO2 mol. (%) @ 25C & 1 Atm': co2_mol,
    'Gas Grav': gas_grav,
    'CR-corrosion defect': corrosion_defect,
    'leak_status': leak_status
})

# Set Timestamp as index for time series analysis
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
df = df.set_index('Timestamp')

# Optional: Save to CSV

df.to_csv('simulated_multiphase_pipeline_time_series_data.csv')
print("\nDataset successfully generated and saved to 'simulated_multiphase_pipeline_time_series_data.csv'")

# Display the class distribution and descriptive statistics
print("Generated Dataset Shape:", df.shape)
print("\nClass Distribution:")
print(df['leak_status'].value_counts())
print("\nDescriptive Statistics for CR-corrosion defect by Leak Status:")
print(df.groupby('leak_status').describe())


Dataset successfully generated and saved to 'simulated_multiphase_pipeline_time_series_data.csv'
Generated Dataset Shape: (10000, 10)

Class Distribution:
leak_status
No Leak       8500
Rupture        860
Large Leak     640
Name: count, dtype: int64

Descriptive Statistics for CR-corrosion defect by Leak Status:
            Wellhead_Temp. (C)                                             \
                         count       mean       std        min        25%   
leak_status                                                                 
Large Leak               640.0  58.097449  1.060455  55.256199  57.390368   
No Leak                 8500.0  61.920764  1.356141  56.853993  60.990633   
Rupture                  860.0  56.284304  1.310014  53.086641  55.448967   

                                             Wellhead_Press (psi)              \
                   50%        75%        max                count        mean   
leak_status                                                 

In [4]:
# Core data science imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Machine learning and preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, learning_curve, StratifiedKFold
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, roc_auc_score, roc_curve, precision_recall_curve, f1_score

# Imbalanced-learn (for SMOTEENN)
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE

# Deep Learning with TensorFlow/Keras and Keras Tuner
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import keras_tuner as kt
from tensorflow.keras import backend as K

# Load the dataset
try:
    df = pd.read_csv('simulated_multiphase_pipeline_data_realistic.csv')
except FileNotFoundError:
    print("Error: The file 'simulated_multiphase_pipeline_data_realistic.csv' was not found.")

# Define the target variable 'leak_status' based on a threshold
threshold = 0.25
df['leak_status'] = np.where(df['CR-corrosion defect'] >= threshold, 'Leak', 'No Leak')

# Display the first few rows to confirm changes
display(df.head())

2025-08-11 11:10:06.952817: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 11:10:06.960218: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 11:10:07.197599: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 11:10:07.631003: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-11 11:10:09.977824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

,Wellhead_Temp. (C),Wellhead_Press (psi),MMCFD- gas,BOPD (barrel of oil produced per day),BWPD (barrel of water produced per day),BSW basic solid and water(%),CO2 mol. (%) @ 25C & 1 Atm,Gas Grav,CR-corrosion defect,leak_status
0,35.527061,420.472210,0.454342,425.997013,3295.175354,2.159792,0.894220,0.690615,0.332707,Leak
1,59.960887,806.263206,2.258694,634.131533,5871.894676,1.030961,2.045793,0.602837,0.054185,No Leak
2,66.742661,471.326311,6.484114,1366.931725,11765.907805,0.657541,0.326901,0.627167,0.037336,No Leak
3,76.122810,404.963830,2.542281,966.376254,5265.419902,1.985121,0.823174,0.663044,0.545875,Leak
4,69.286778,603.067748,2.386041,608.994706,3348.992092,1.499423,0.343359,0.637470,0.042304,No Leak


In [5]:
le = LabelEncoder()
y = le.fit_transform(df['leak_status'])
X = df.drop(['CR-corrosion defect', 'leak_status'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

In [6]:
# Reshape the data into sequences for the LSTM
SEQUENCE_LENGTH = X_train.shape[1]

# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for LSTM
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], SEQUENCE_LENGTH, 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], SEQUENCE_LENGTH, 1)

# --- Custom Keras F1-score Metric for use in model.compile ---
def f1_score_keras(y_true, y_pred):
    y_true = tf.cast(y_true, 'float32')
    y_pred = tf.cast(y_pred, 'float32')
    y_pred_rounded = tf.round(y_pred)
    
    true_positives = tf.reduce_sum(y_true * y_pred_rounded)
    possible_positives = tf.reduce_sum(y_true)
    predicted_positives = tf.reduce_sum(y_pred_rounded)

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1

# --- Custom function for plotting learning curves ---
def plot_keras_learning_curve_no_resample(model_builder, X, y, title, cv=5):
    f1_scores_train = []
    f1_scores_val = []

    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    for train_index, val_index in skf.split(X, y):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]
        
        X_train_fold = X_train_fold.reshape(X_train_fold.shape[0], SEQUENCE_LENGTH, 1)
        X_val_fold = X_val_fold.reshape(X_val_fold.shape[0], SEQUENCE_LENGTH, 1)

        model = model_builder()
        history = model.fit(X_train_fold, y_train_fold,
                            validation_data=(X_val_fold, y_val_fold),
                            epochs=20, 
                            batch_size=32,
                            verbose=0)
        
        y_pred_proba_val = model.predict(X_val_fold)
        # Use optimal threshold for evaluation
        precision, recall, thresholds = precision_recall_curve(y_val_fold, y_pred_proba_val)
        f1_scores_val_all = (2 * precision * recall) / (precision + recall)
        f1_scores_val_all[np.isnan(f1_scores_val_all)] = 0
        optimal_threshold_idx = np.argmax(f1_scores_val_all)
        optimal_threshold = thresholds[optimal_threshold_idx]
        y_pred_val = (y_pred_proba_val > optimal_threshold).astype("int32")

        val_score = f1_score(y_val_fold, y_pred_val)
        f1_scores_val.append(val_score)

        y_pred_proba_train = model.predict(X_train_fold)
        y_pred_train = (y_pred_proba_train > optimal_threshold).astype("int32")
        train_score = f1_score(y_train_fold, y_pred_train)
        f1_scores_train.append(train_score)
        
    print(f"F1 Scores on CV folds: Train: {np.mean(f1_scores_train):.4f}, Validation: {np.mean(f1_scores_val):.4f}")

    plt.figure(figsize=(10, 7))
    plt.plot([0, 1], [np.mean(f1_scores_train), np.mean(f1_scores_train)], 'o-', color='r', label='Average Training Score')
    plt.plot([0, 1], [np.mean(f1_scores_val), np.mean(f1_scores_val)], 'o-', color='g', label='Average Cross-Validation Score')
    plt.title(title)
    plt.xlabel('Training Examples (Simulated)')
    plt.ylabel('F1 Score')
    plt.legend(loc='best')
    plt.grid()
    plt.show()


In [ ]:
print("--- Training Baseline LSTM Model ---")

def build_baseline_model():
    model = Sequential()
    model.add(Input(shape=(SEQUENCE_LENGTH, 1)))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

baseline_model = build_baseline_model()
history_baseline = baseline_model.fit(X_train_reshaped, y_train,
                                      epochs=50, batch_size=32,
                                      validation_split=0.2,
                                      callbacks=[EarlyStopping(monitor='val_loss', patience=10)],
                                      verbose=0)

print("\n--- Evaluating Baseline Model on Test Set (With Optimal Threshold) ---")
y_pred_proba_baseline = baseline_model.predict(X_test_reshaped)

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_baseline)
f1_scores_test = (2 * precision * recall) / (precision + recall)
f1_scores_test[np.isnan(f1_scores_test)] = 0
optimal_threshold_idx = np.argmax(f1_scores_test)
optimal_threshold_baseline = thresholds[optimal_threshold_idx]
print(f"Optimal threshold for baseline model: {optimal_threshold_baseline:.4f}")

y_pred_baseline = (y_pred_proba_baseline > optimal_threshold_baseline).astype("int32")

print(classification_report(y_test, y_pred_baseline, target_names=le.classes_))
avg_f1_baseline = f1_score(y_test, y_pred_baseline)
print(f"Average F1 Score on Test Set: {avg_f1_baseline:.4f}")

ConfusionMatrixDisplay.from_predictions(y_test, y_pred_baseline, display_labels=le.classes_, cmap=plt.cm.Blues)
plt.title("Confusion Matrix: Baseline Model")
plt.show()

print("\n--- Learning Curve: Baseline Model ---")
plot_keras_learning_curve_no_resample(build_baseline_model, X_train_scaled, y_train, 'Learning Curve: Baseline Model (No Resampling)')

